In [ ]:
import nltk
import difflib
import time
import gc
import itertools
import multiprocessing
import pandas as pd
import numpy as np
import xgboost as xgb
import lightgbm as lgb

import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split, StratifiedKFold

from models_utils_fe import *
from models_utils_skf import *

In [ ]:
def load_oof(mode = 'train'):
    src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/OOF_preds/'
    oof_preds = pd.DataFrame()
    files = sorted([x for x in os.listdir(src + '{}/'.format(mode)) if '.pkl' in x])
    print('Loading OOF preds:', files)
    for i in files:
        df_preds = pd.read_pickle('{}/{}/{}'.format(src, mode, i))
        oof_preds = pd.concat([oof_preds, df_preds], axis = 1)
    oof_preds.columns = ['{}_OOFpreds'.format(i) for i in range(oof_preds.shape[1])]
    return oof_preds

In [ ]:
src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/features/'
feats_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/data/features/uncleaned/'
oof_src = '/media/w/1c392724-ecf3-4615-8f3c-79368ec36380/DS Projects/Kaggle/Quora/scripts/models/OOF_preds/train/'

X_train = pd.read_pickle('Xtrain_814colsBest.pkl', compression = 'bz2')
xgb_feats = pd.read_csv(feats_src + '/the_1owl/owl_train.csv')
y_train = xgb_feats[['is_duplicate']]

train_interactions = pd.read_pickle(src + 'train_tony_interaction_gru.pkl')
oof_train = load_oof()

X_train = pd.concat([X_train, train_interactions, oof_train.iloc[:, 3:]], axis = 1)
X_train = X_train.astype('float32')

del xgb_feats, oof_train, train_interactions
gc.collect()

LGBM: Final score for fold 1 : 0.180640621299 

In [ ]:
lgb_params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'binary',
    'metric' : {'binary_logloss'},
    'learning_rate' : 0.03,
    'feature_fraction' : 0.51,
    'bagging_fraction': 0.9,
    'bagging_freq': 100,
    'num_leaves' : 255,
    'max_depth': 8,
    'min_data_in_leaf': 23,
    'subsample': 0.8,
    'colsample_bytree': 0.41,
    'silent': 1,
    'random_state': 1337,
    'verbose': 1,
    'nthread': 9,
}

xgb_params = {
    'seed': 1337,
    'colsample_bytree': 0.42,
    'silent': 1,
    'subsample': 0.85,
    'eta': 0.02,
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 8,
    'min_child_weight': 20,
    'nthread': 4,
    'tree_method': 'hist',
    }

In [ ]:
lgb_foldrun(X_train, y_train, lgb_params, 'BestGRU_experiments')

In [ ]:
gbm = xgb.Booster(model_file = 'saved_models/XGB/XGB_10SKF_FredFeatsGRU_loss0.17917_fold1.txt')
dtrain = xgb.DMatrix(X_train, label = y_train)

mapper = {'f{0}'.format(i): v for i, v in enumerate(dtrain.feature_names)}
importance = {mapper[k]: v for k, v in gbm.get_fscore().items()}
importance = sorted(importance.items(), key=lambda x:x[1], reverse=True)[:20]

df_importance = pd.DataFrame(importance, columns=['feature', 'fscore'])
df_importance['fscore'] = df_importance['fscore'] / df_importance['fscore'].sum()

plt.figure()
df_importance.plot()
df_importance.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(10, 18))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')

In [ ]:
df_importance.feature.tolist()